In [ ]:
#Name: Abel
#Date: 2021/07/27
#Purpose: To learn the basics to using Selenium for dynamic web scraping & scraping car images off of google images
#Reference: https://chromedriver.chromium.org/getting-started & https://www.analyticsvidhya.com/blog/2020/08/web-scraping-selenium-with-python/

import time
from selenium import webdriver

#--------------from Analyticsvidhya-------------------
import os
import io
from PIL import Image
import requests

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('http://images.google.com/');
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('car')
search_box.submit()
time.sleep(3) # Let the user actually see something!

#-------------------------analyticsvidhya begins-----------------
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)#sleep_between_interactions

imgResults = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
totalResults=len(imgResults)
print (totalResults)

#Click on each Image to extract its corresponding link to download

img_urls = set()
for i in range(0,int(len(imgResults)/10)):
    img=imgResults[i]
    try:
        img.click()
        time.sleep(2)
        actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
        for actual_image in actual_images:
            if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                img_urls.add(actual_image.get_attribute('src'))
    except ElementClickInterceptedException or ElementNotInteractableException as err:
        print(err)


#Download & save each image in the Destination directory
os.chdir('D:/Temp/Courses/Python/WebScraping')
baseDir=os.getcwd()

for i, url in enumerate(img_urls):
    file_name = f"{i}.jpg"    
    try:
        image_content = requests.get(url).content
    except Exception as e:
            print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = os.path.join(baseDir, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

driver.quit() # closes the chrome browswer